# 简单RAG简介

检索增强生成（Retrieval-Augmented Generation，RAG）是一种将信息检索与生成式模型结合的混合方法。它通过引入外部知识提升语言模型的表现，从而提高准确性和事实性。

在简单RAG流程中，我们遵循以下步骤：

1. **数据导入**：加载并预处理文本数据。
2. **分块**：将数据拆分为更小的片段，以提升检索效果。
3. **嵌入生成**：使用嵌入模型将文本片段转换为数值表示。
4. **语义检索**：根据用户查询检索相关片段。
5. **响应生成**：基于检索到的文本，使用语言模型生成回答。

本Notebook实现了简单RAG方法，对模型响应进行评估，并探索多种改进方式。

## 环境准备
我们首先导入必要的库。

In [2]:
import fitz
import os
import numpy as np
import json
from openai import OpenAI

In [3]:
# 从.env文件中加载环境变量
from dotenv import load_dotenv
load_dotenv()

True

## 从PDF文件中提取文本
为了实现RAG，我们首先需要文本数据源。本例中，我们使用PyMuPDF库从PDF文件中提取文本。

In [ ]:
def extract_text_from_pdf(pdf_path):
    """
    从PDF文件中提取文本，并打印前num_chars个字符。

    参数:
    pdf_path (str): PDF文件路径。

    返回:
    str: 提取的文本内容。
    """
    # 打开PDF文件
    mypdf = fitz.open(pdf_path)
    all_text = ""  # 初始化用于存储提取文本的字符串

    # 遍历PDF的每一页
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]  # 获取当前页
        text = page.get_text("text")  # 提取当前页的文本
        all_text += text  # 将提取的文本追加到all_text中

    return all_text  # 返回提取的文本

## 对提取的文本进行分块
获得提取的文本后，我们将其划分为较小且有重叠的片段，以提升检索准确性。

In [ ]:
def chunk_text(text, n, overlap):
    """
    将文本分割为长度为n、重叠为overlap的片段。

    参数:
    text (str): 需要分块的文本。
    n (int): 每个片段的字符数。
    overlap (int): 相邻片段之间的重叠字符数。

    返回:
    List[str]: 分块后的文本列表。
    """
    chunks = []  # 初始化用于存储片段的列表
    
    # 以步长(n - overlap)遍历文本
    for i in range(0, len(text), n - overlap):
        # 从i到i+n切片并加入chunks
        chunks.append(text[i:i + n])

    return chunks  # 返回分块后的文本列表

## 配置OpenAI API客户端
我们初始化OpenAI客户端，用于生成嵌入和回答。

In [ ]:
# 使用基础URL和API密钥初始化OpenAI客户端
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 从环境变量获取API密钥
)

## 从PDF文件提取并分块文本
现在，我们加载PDF，提取文本并将其分块。

In [9]:
# 定义PDF文件路径
pdf_path = "data/AI_Information.pdf"

# 从PDF文件中提取文本
extracted_text = extract_text_from_pdf(pdf_path)

# 将提取的文本分割为长度1000、重叠200的片段
text_chunks = chunk_text(extracted_text, 1000, 200)

# 打印分块数量
print("Number of text chunks:", len(text_chunks))

# 打印第一个文本片段
print("\nFirst text chunk:")
print(text_chunks[0])

Number of text chunks: 42

First text chunk:
Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths and fiction. 
However, the formal field of AI research began in the mid-20th century. The Dartmouth Workshop 
in 1956 is widely considered the birthplace of AI. Early AI research focused on problem-solving 
and 

## 为文本片段生成嵌入
嵌入将文本转换为数值向量，便于高效的相似度检索。

In [ ]:
def create_embeddings(text, model="BAAI/bge-en-icl"):
    """
    使用指定的OpenAI模型为文本生成嵌入。

    参数:
    text (str): 需要生成嵌入的文本。
    model (str): 用于生成嵌入的模型，默认为"BAAI/bge-en-icl"。

    返回:
    dict: 包含嵌入的OpenAI API响应。
    """
    # 使用指定模型为输入文本生成嵌入
    response = client.embeddings.create(
        model=model,
        input=text
    )

    return response  # 返回包含嵌入的响应

# 为文本片段生成嵌入
response = create_embeddings(text_chunks)

## 执行语义检索
我们实现余弦相似度算法，以根据用户查询找到最相关的文本片段。

In [ ]:
def cosine_similarity(vec1, vec2):
    """
    计算两个向量的余弦相似度。

    参数:
    vec1 (np.ndarray): 第一个向量。
    vec2 (np.ndarray): 第二个向量。

    返回:
    float: 两个向量的余弦相似度。
    """
    # 计算两个向量的点积并除以它们的范数乘积
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [ ]:
def semantic_search(query, text_chunks, embeddings, k=5):
    """
    使用查询和嵌入对文本片段进行语义检索。

    参数:
    query (str): 检索用的查询语句。
    text_chunks (List[str]): 待检索的文本片段列表。
    embeddings (List[dict]): 文本片段的嵌入列表。
    k (int): 返回最相关片段的数量，默认为5。

    返回:
    List[str]: 前k个最相关的文本片段。
    """
    # 为查询生成嵌入
    query_embedding = create_embeddings(query).data[0].embedding
    similarity_scores = []  # 初始化用于存储相似度分数的列表

    # 计算查询嵌入与每个文本片段嵌入的相似度
    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding.embedding))
        similarity_scores.append((i, similarity_score))  # 记录索引和相似度分数

    # 按相似度分数降序排序
    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    # 获取前k个最相似文本片段的索引
    top_indices = [index for index, _ in similarity_scores[:k]]
    # 返回前k个最相关的文本片段
    return [text_chunks[index] for index in top_indices]


## 在提取的片段上运行查询

In [ ]:
# 从JSON文件加载验证数据
with open('data/val.json') as f:
    data = json.load(f)

# 获取验证数据中的第一个查询
query = data[0]['question']

# 执行语义检索，获取与查询最相关的前2个文本片段
top_chunks = semantic_search(query, text_chunks, response.data, k=2)

# 打印查询内容
print("Query:", query)

# 打印前2个最相关的文本片段
for i, chunk in enumerate(top_chunks):
    print(f"Context {i + 1}:\n{chunk}\n=====================================")

## 基于检索到的片段生成回答

In [ ]:
# 定义AI助手的系统提示词
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, user_message, model="meta-llama/Llama-3.2-3B-Instruct"):
    """
    基于系统提示词和用户消息生成AI模型的回答。

    参数:
    system_prompt (str): 指导AI行为的系统提示词。
    user_message (str): 用户消息或查询。
    model (str): 用于生成回答的模型，默认为"meta-llama/Llama-2-7B-chat-hf"。

    返回:
    dict: AI模型的响应。
    """
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
    )
    return response

# 基于top_chunks生成用户提示词
user_prompt = "\n".join([f"Context {i + 1}:\n{chunk}\n=====================================\n" for i, chunk in enumerate(top_chunks)])
user_prompt = f"{user_prompt}\nQuestion: {query}"

# 生成AI回答
ai_response = generate_response(system_prompt, user_prompt)

## 评估AI回答
我们将AI的回答与期望答案进行对比，并给出评分。

In [ ]:
# 定义评估系统的系统提示词
evaluate_system_prompt = "You are an intelligent evaluation system tasked with assessing the AI assistant's responses. If the AI assistant's response is very close to the true response, assign a score of 1. If the response is incorrect or unsatisfactory in relation to the true response, assign a score of 0. If the response is partially aligned with the true response, assign a score of 0.5."

# 组合用户查询、AI回答、真实答案和评估系统提示词，生成评估提示词
evaluation_prompt = f"User Query: {query}\nAI Response:\n{ai_response.choices[0].message.content}\nTrue Response: {data[0]['ideal_answer']}\n{evaluate_system_prompt}"

# 使用评估系统提示词和评估提示词生成评估结果
evaluation_response = generate_response(evaluate_system_prompt, evaluation_prompt)

# 打印评估结果
print(evaluation_response.choices[0].message.content)

